# ArangoDB NetworkX Adapter Getting Started Guide  

<a href="https://colab.research.google.com/github/arangoml/networkx-adapter/blob/master/examples/ArangoDB_NetworkxAdapter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![networkX](https://github.com/arangoml/networkx-adapter/blob/master/examples/assets/logos/networkx_logo.svg?raw=1) 
![arangodb](https://github.com/arangoml/networkx-adapter/blob/master/examples/assets/logos/ArangoDB_logo.png?raw=1) 

Version: 0.0.0.2.5.3

In this Notebook we learn how to export Graphs from [ArangoDB](https://www.arangodb.com/), a multi-model Graph Database into [NetworkX](https://networkx.github.io/), the swiss army knife for graph analysis ion python.

# Setup

In [ ]:
%%capture
!git clone -b oasis_connector --single-branch https://github.com/arangodb/interactive_tutorials.git
!git clone -b 0.0.0.2.5.3 --single-branch https://github.com/arangoml/networkx-adapter.git
!rsync -av networkx-adapter/examples/ ./ --exclude=.git
!rsync -av interactive_tutorials/ ./ --exclude=.git
!pip3 install adbnx_adapter=0.0.0.2.5.3
!pip3 install networkx
!pip3 install matplotlib
!pip3 install pyarango
!pip3 install python-arango

In [ ]:
import json
import oasis
import networkx as nx
import matplotlib.pyplot as plt


from adbnx_adapter.arangoDB_networkx_adapter import ArangoDB_Networkx_Adapter

# Create a Temporary ArangoDB Instance

In [ ]:
# Request temporary instance from the managed ArangoDB Cloud Oasis.
con = oasis.getTempCredentials()

# Connect the driver to the temp database
conn = oasis.connect(con)
db = conn[con["dbName"]]

print()
print("https://{}:{}".format(con["hostname"], con["port"]))
print("Username: " + con["username"])
print("Password: " + con["password"])
print("Database: " + con["dbName"])

Feel free to use to above URL to checkout the UI!

# Data Import

We will use an Fraud Detection example graph, explained in more detail in this [interactive notebook](https://colab.research.google.com/github/joerg84/Graph_Powered_ML_Workshop/blob/master/Fraud_Detection.ipynb).

*Note the included arangorestore will only work on Linux system, if you want to run this notebook on a different OS please consider using the appropriate arangorestore from the [Download area](https://www.arangodb.com/download-major/).*

In [ ]:
!chmod -R 755 ./tools
!./tools/arangorestore -c none --server.endpoint http+ssl://{con["hostname"]}:{con["port"]} --server.username {con["username"]} --server.database {con["dbName"]} --server.password {con["password"]} --default-replication-factor 3  --input-directory "data/fraud_dump"

# Create Graph

The graph we will be using in the following looks as follows:

![networkX](https://github.com/arangoml/networkx-adapter/blob/master/examples/assets/fraud_graph.jpeg?raw=1) 

In [ ]:
from pyArango.collection import Collection, Edges, Field
from pyArango.graph import Graph, EdgeDefinition


class account(Collection):
    _fields = {
        "Name": Field()
    }
    
class customer(Collection):
    _fields = {
        "Name": Field()
    }
    
class transaction(Edges): 
    _fields = {
    }

class accountHolder(Edges): 
    _fields = {
    }

class FraudDetection(Graph) :
    _edgeDefinitions = [EdgeDefinition("accountHolder", fromCollections=["customer"], toCollections=["account"]),EdgeDefinition("transaction", fromCollections=["account"], toCollections=["account"])]
    _orphanedCollections = []

fraudGraph = db.createGraph("FraudDetection")

print("Collection/Graph Setup done.")

Feel free to visit the ArangoDB UI using the above link and login data and check the Graph!

# Connect ArangoDB and NetworkX 

We first connect the ArangoDB_Networkx_Adapter to our temp ArangoDB cluster:

In [ ]:
ma = ArangoDB_Networkx_Adapter(conn = con)

Next, we need to define the attributes in the vertex and edge collections to be included:

*Note, we are currently working on making this step optional in the future!*

In [ ]:
fraud_detection_attributes = { 'vertexCollections': 
                                  {'account': {'Balance', 'account_type', 'customer_id', 'rank'},\
                                   'bank': {'Country', 'Id', 'bank_id', 'bank_name'},\
                                   'branch':{'City', 'Country', 'Id', 'bank_id', 'branch_id', 'branch_name'},\
                                    'Class':{'concrete', 'label', 'name'},\
                                    'customer': {'Name', 'Sex', 'Ssn', 'rank'}},\
                               'edgeCollections' : 
                                  {'accountHolder': {'_from', '_to'},\
                                   'Relationship': {'_from', '_to', 'label', 'name', 'relationshipType'},\
                                    'transaction': {'_from', '_to'}}}

Now, we can export the networkX graph:

In [ ]:
g = ma.create_networkx_graph(graph_name = 'FraudDetection',  graph_attributes = fraud_detection_attributes)

From here on we can simply use all networkX functionality:

In [ ]:
nx.draw(g, with_labels=True)

In [ ]:
g.nodes()

In [ ]:
g.edges()

# Next Steps

* Explore [Interactive NetworkX Notebook](https://colab.research.google.com/github/joerg84/Graph_Powered_ML_Workshop/blob/master/NetworkX.ipynb)
* Explore [ArangoDB](https://www.arangodb.com/)
* Explore [Interactive ArangoDB tutorials](https://github.com/joerg84/ArangoDBUniversity#arangodb-university)